In [6]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'digit-recognizer:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3004%2F861823%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240405%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240405T060255Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D68f321a63893a15b7592170347c91a9ff2f5c7e8f3aab843d9a7834e1f31871a80b9169f3a39acf619d36b316a82118c130a24fa86d5282ee7ce1002f34a74627c05a0bda8fae943306c894851a3a696a38f430e8d94618d5a1f76c7f64e3de77f87f9aa2e9516a97b7e161b26e6fc349028e4a4db3e7e45fc614c56fb31795868d14083a7f362f6924c6a90a0de55e8f26838bd22f043b6fea8f89b0da28695f4182909ff15ac262da12527dcdc56bf6ef7c43b1c7d1de265e6c328fe4a6ba901eba82a57c2acb06fff6705c69d889b84268c9e21a8f53749047ce2043f20b519212a9d112c6b8bbb4fc267b749013d43fe6a6cd6e656f388cca36c9ba3dfcf'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 16054568 bytes downloaded
Downloaded and uncompressed: digit-recognizer
Data source import complete.


## <center>Handwritten Digit Recognition (HDR) using Convolutional Neural Network (CNN) with Tensorflow</center>
<center> <b>A Deep Learning Analysis with Real World Data</b> </center>

![digits_recognition_cnn.png](attachment:ab119fe8-5a50-4785-93a0-058d6a932b32.png)

<a class="anchor" id="bcImp"></a>
## <span style='color:rgb(205, 0, 153)'>Table of Contents</span>
* [<b>Abstract</b>](#abstract)
* [<b>Import the dependencies and load the dataset</b>](#importData)
* [<b>Data overview</b>](#dOverview)
     * [Dimension of train and test data](#dTrainTest)
     * [Visualizing the data using TSNE](#visualizeTSNE)
     * [Splitting data into training and validation dataset](#splitData)
     * [Dimension of training and validation data](#dTrainVal)
     * [Converting training, testing, and validation data into array](#reTrainTestVal)
     * [Dimension of training, testing, and validation data after reshape](#dreshape)
     * [<b>In summary</b>](#summaryData)
* [<b>Explore the data</b>](#exploreData)
     * [Visualise how the digits were written](#visualizeData)
     * [Reshaping train, test, and validation data](#reshapeData)
     * [Normalize train, test, and validation data](#normalizeData)
     * [<b>In summary</b>](#summaryExploreData)
* [<b>Build the CNN model to Classify Handwritten Digits</b>](#modeling)
<a class="anchor" id="modelAnchor"></a>
     * [Summary of the training model](#modelSummary)
     * [Visualization of the model](#modelplot)
     * [Compile the model using keras.optimizers.Adam](#compileModel)
     * [Train the model](#trainModel)
     * [<b>In summary</b>](#modelBuildSummary)
* [<b>Model evaluation</b>](#modelEvaluation)
<a class="anchor" id="bcImp1"></a>
     * [Loss plot curve for training and validation dataset](#lossPlot)
     * [Accuracy plot curve for training and validation dataset](#accuracyPlot)
     * [<b>Evaluation of the model accuracy</b>](#accuracyEvaluation)
         * [Performance of training dataset](#perfTrain)
         * [Performance of validation dataset](#perfValidation)
         * [Save and load the model](#saveModel)
         * [Visualise validation predicted data on how the digits were written](#visualizevalidation)
         * [Confusion matrix of validation dataset](#cmValidation)
     * [<b>In summary</b>](#modelEvaluationSummary)
* [<b>Model prediction on unseen dataset (test data)</b>](#modelprediction)
     * [Visualise test predicted data how the digits were written](#visualizePredict)
* [<b>Submission</b>](#submission)

## <span style='color:rgb(205, 0, 153)'>So, let's get started</span> 🧑👈🙏💪

## <span style='color:rgb(205, 0, 153)'>1 | Abstract </span><a class="anchor" id="abstract"></a>
<hr>

### <b>Introduction</b>
Handwritten Digit Recognition is the process of digitizing human handwritten digit images. It is a difficult task for the machine because handwritten digits are not perfect and can be made with a variety of flavors. In order to address this issue, we created HDR, which uses the image of a digit to identify the digit that is present in the image.

In this project, we developed a [Convolutional Neural Network](https://en.wikipedia.org/wiki/Convolutional_neural_network) <code>(CNN)</code> model using <b>Tensorflow framework</b> to <b>Recognition of Handwritten Digit</b>.

A convolutional neural network (CNN, or ConvNet) is a Deep Learning algorithm that can take in an input image, assign learnable weights and biases to various objects in the image and be able to distinguish one from the other.

It is used to analyse visual imagery. <code>Object detection</code>, <code>face recognition</code>, <code>robotics</code>, <code>video analysis</code>, <code>segmentation</code>, <code>pattern recognition</code>, <code>natural language processing</code>, <code>spam detection</code>, <code>topic categorization</code>, <code>regression analysis</code>, <code>speech recognition</code>, <code>image classification</code> are some of the examples that can be done using Convolutional Neural Networking.
### <b>Approach</b>
We have used Sequential Keras model which has two pairs of <code>Convolution2D</code> and <code>MaxPooling2D</code> layers. The MaxPooling layer acts as a sort of downsampling using max values in a region instead of averaging. After that we will use Flatten layer to convert multidimensional parameters to vector.

The last layer has a Dense layer with 10 Softmax outputs. The output represents the network guess. The 0-th output represents a probability that the input digit is 0, the 1-st output represents a probability that the input digit is 1 and so on...
#### <b>Result</b>
CNN performed well, providing validation accuracy and loss score of <code>98.9%%</code> and <code>4.5%</code> respectively.
#### <b>Conclusion</b>
Convolutional neural network (CNN, or ConvNet) can be used to predict Handwritten Digits reasonably. We have successfully developed Handwritten digit recognition with Python, Tensorflow, and Machine Learning libraries. Handwritten Digits have been recognized by more than <code>98.9%</code> validation accuracy.

<b>Keywords</b> – convolutional neural network, mnist, deep learning, handwritten digits recognition.

## <span style='color:rgb(205, 0, 153)'>2 | Import the dependencies and load the dataset </span> <a class="anchor" id="importData"></a>
<hr>

[Back to Table of Contents](#bcImp)

We will import all of the modules that we will require to train our model.

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import datetime
import platform

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/train.csv


In [8]:
print('Python version:', platform.python_version())
print('Tensorflow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)

Python version: 3.10.12
Tensorflow version: 2.15.0


AttributeError: module 'keras.api._v2.keras' has no attribute '__version__'

In [ ]:
# Load the TensorBoard notebook extension.
# %reload_ext tensorboard
%load_ext tensorboard

In [ ]:
# Clear any logs from previous runs.
!rm -rf ./.logs/

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

## <span style='color:rgb(205, 0, 153)'> 3 | Data Overview </span><a class="anchor" id="dOverview"></a>
<hr>

[Back to Table of Contents](#bcImp)

The __MNIST dataset__ contains <code>42,000</code> training images of handwritten digits from __zero to nine (10 different classes)__ and <code>28,000</code> images for testing without the label identifier (for submission). These images are the grayscaled pixel value and are represented as a <code>28×28</code> matrix.

### 3.1 | Dimension of train and test data <a class="anchor" id="dTrainTest"></a>
[Back to Table of Contents](#bcImp)

In [ ]:
train.head()

In [ ]:
train.info(), train.shape

In [ ]:
test.info(),  test.shape

In [ ]:
X = train.iloc[:, 1:785]
y = train.iloc[:, 0]

X_test = test.iloc[:, 0:784]

### <span style='color:rgb(205, 0, 153)'>3.2 | Visualizing the data using TSNE </span><a class="anchor" id="visualizeTSNE"></a>
<hr>

[Back to Table of Contents](#bcImp)

<b>TSNE</b> - t-Distributed Stochastic Neighbor embedding. This is a dimensionality reduction algorithm that is designed to keep local structure in the high dimensional data set, but cares less about global structure. Here, we use it to go from the 784 pixel-dimension of the images to two dimensions. This makes plotting easier. The color scale is the original MNIST label and one can see that the separation of the labels is apparent.

In [ ]:
# WARNING: running t-SNE on the full data set takes a while.
X_tsn = X/255

from sklearn.manifold import TSNE
tsne = TSNE()

tsne_res = tsne.fit_transform(X_tsn)

In [ ]:
plt.figure(figsize=(14, 12))
plt.scatter(tsne_res[:,0], tsne_res[:,1], c=y, s=2)
plt.xticks([])
plt.yticks([])
plt.colorbar();

### 3.3 | Splitting data into training and validation dataset<a class="anchor" id="splitData"></a>
[Back to Table of Contents](#bcImp)

We are dividing our dataset __(X)__ into two parts.
<ol>
    <li> The <b>training dataset (80%)</b> is used to fit our models</li>
    <li>The <b>Validation dataset (20%)</b> is used to evaluate our models</li>
 </ol>

<code>train_test_split()</code> method returns us the training data, its labels and also the validation data and its labels.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2,random_state = 1212)

### 3.4 | Dimension of training and validation data<a class="anchor" id="dTrainVal"></a>
[Back to Table of Contents](#bcImp)

In [ ]:
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_validation:', X_validation.shape)
print('y_validation:', y_validation.shape)

### 3.5 | Converting training, testing, and validation data into array<a class="anchor" id="reTrainTestVal"></a>
[Back to Table of Contents](#bcImp)

In [ ]:
x_train_re = X_train.to_numpy().reshape(33600, 28, 28)
y_train_re = y_train.values
x_validation_re = X_validation.to_numpy().reshape(8400, 28, 28)
y_validation_re = y_validation.values
x_test_re = test.to_numpy().reshape(28000, 28, 28)

### 3.6 | Dimension of training, testing, and validation data after reshaping<a class="anchor" id="dreshape">
[Back to Table of Contents](#bcImp)

In [ ]:
print('x_train:', x_train_re.shape)
print('y_train:', y_train_re.shape)
print('x_validation:', x_validation_re.shape)
print('y_validation:', y_validation_re.shape)
print('x_test:', x_test_re.shape)

In [ ]:
# Save image parameters to the constants that we will use later for data re-shaping and for model traning.
(_, IMAGE_WIDTH, IMAGE_HEIGHT) = x_train_re.shape
IMAGE_CHANNELS = 1

print('IMAGE_WIDTH:', IMAGE_WIDTH);
print('IMAGE_HEIGHT:', IMAGE_HEIGHT);
print('IMAGE_CHANNELS:', IMAGE_CHANNELS);

 ### In summary <a class ="anchor" id ="summaryData"></a>
 [Back to Table of Contents](#bcImp)

## <span style='color:rgb(205, 0, 153)'> 4 | Explore the data </span><a class="anchor" id="exploreData">
[Back to Table of Contents](#bcImp)
    
Here is how each image in the dataset looks like. It is a <code>28x28</code> matrix of integers (from 0 to 255) and each integer represents a color of a pixel.

In [ ]:
pd.DataFrame(x_train_re[0])

### 4.1 | Visualise how the digits were written <a class ="anchor" id="visualizeData"></a>
[Back to Table of Contents](#bcImp)

This number matrix may be drawn as follows:

In [ ]:
plt.imshow(x_train_re[0], cmap=plt.cm.binary)
plt.show()

Our first record of training data represents 2.

Let's print some more training examples to get the feeling of how the digits were written.

In [ ]:
numbers_to_display = 100
num_cells = math.ceil(math.sqrt(numbers_to_display))
plt.figure(figsize=(20,20))
for i in range(numbers_to_display):
    plt.subplot(num_cells, num_cells, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train_re[i], cmap=plt.cm.binary)
    plt.xlabel(y_train_re[i])
plt.show()

### 4.2 | Reshaping train, test, and validation data <a class ="anchor" id = "reshapeData"></a>
[Back to Table of Contents](#bcImp)

In order to use convolution layers we need to reshape our data and add a color channel to it. As you've noticed currently every digit has a shape of (28, 28) which means that it is a 28x28 matrix of color values form 0 to 255. We need to reshape it to (28, 28, 1) shape so that each pixel potentially may have multiple channels (like Red, Green and Blue).

In [ ]:
x_train_with_chanels = x_train_re.reshape(
    x_train_re.shape[0],
    IMAGE_WIDTH,
    IMAGE_HEIGHT,
    IMAGE_CHANNELS
)

x_validation_with_chanels = x_validation_re.reshape(
    x_validation_re.shape[0],
    IMAGE_WIDTH,
    IMAGE_HEIGHT,
    IMAGE_CHANNELS
)

x_test_with_chanels = x_test_re.reshape(
    x_test_re.shape[0],
    IMAGE_WIDTH,
    IMAGE_HEIGHT,
    IMAGE_CHANNELS
)

In [ ]:
print('x_train_with_chanels:', x_train_with_chanels.shape)
print('x_validation_with_chanels:', x_validation_with_chanels.shape)
print('x_test_with_chanels:', x_test_with_chanels.shape)

### 4.3 | Normalize train, test, and validation data <a class ="anchor" id ="normalizeData"></a>
[Back to Table of Contents](#bcImp)

Here we're just trying to normalize from values range of [0...255] to [0...1].

In [ ]:
x_train_normalized = x_train_with_chanels / 255
x_validation_normalized = x_validation_with_chanels / 255
x_test_normalized = x_test_with_chanels / 255

In [ ]:
# Let's check just one row from the 0th image to see color chanel values after normalization.
x_train_normalized[0][10]

### In summary<a class ="anchor" id ="summaryExploreData"></a>
[Back to Table of Contents](#bcImp)

## <span style='color:rgb(205, 0, 153)'> 5 | Build the CNN model to Classify Handwritten Digits </span><a class = "anchor" id = "modeling"></a>
<hr>

[Back to Table of Contents](#modelAnchor)

A Convolutional Neural Network model generally consists of convolutional and pooling layers.

We are using Sequential Keras model which have two pairs of Convolution2D and MaxPooling2D layers. The MaxPooling layer acts as a sort of downsampling using max values in a region instead of averaging.

After that we will use Flatten layer to convert multidimensional parameters to vector.

The last layer will be a Dense layer with 10 Softmax outputs. The output represents the network guess. The 0-th output represents a probability that the input digit is 0, the 1-st output represents a probability that the input digit is 1 and so on...

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Convolution2D(
    input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
    kernel_size=5,
    filters=8,
    strides=1,
    activation=tf.keras.activations.relu,
    kernel_initializer=tf.keras.initializers.VarianceScaling()
))

model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

model.add(tf.keras.layers.Convolution2D(
    kernel_size=5,
    filters=16,
    strides=1,
    activation=tf.keras.activations.relu,
    kernel_initializer=tf.keras.initializers.VarianceScaling()
))

model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(
    units=128,
    activation=tf.keras.activations.relu
));

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(
    units=10,
    activation=tf.keras.activations.softmax,
    kernel_initializer=tf.keras.initializers.VarianceScaling()
))

### 5.1 | Summary of the training model <a class ="anchor" id = "modelSummary"></a>
[Back to Table of Contents](#modelAnchor)

Here is our model summary so far.

In [ ]:
model.summary()

### 5.2 | Visualization of the model <a class ="anchor" id = "modelplot"></a>
[Back to Table of Contents](#modelAnchor)

In order to plot the model the graphviz should be installed.

A model summary that describes the various layers defined in the model.

In [ ]:
tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
)

### 5.3 | Compile the model using keras.optimizers.Adam <a class ="anchor" id = "compileModel"></a>
[Back to Table of Contents](#modelAnchor)

In [ ]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=adam_optimizer,
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

### 5.4 | Train the model <a class ="anchor" id ="trainModel"></a>
[Back to Table of Contents](#modelAnchor)

In [ ]:
log_dir=".logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

training_history = model.fit(
    x_train_normalized,
    y_train_re,
    epochs=10,
    validation_data=(x_validation_normalized, y_validation_re),
    callbacks=[tensorboard_callback]
)

print("The model has successfully trained")

### In summary<a class ="anchor" id ="modelBuildSummary"></a>
[Back to Table of Contents](#bcImp)

## <span style='color:rgb(205, 0, 153)'> 6 | Model evaluation </span><a class ="anchor" id ="modelEvaluation"></a>
<hr>

[Back to Table of Contents](#modelAnchor)

### 6.1 | Loss plot curve for training and validation<a class="anchor" id ="lossPlot"><a/>
[Back to Table of Contents](#bcImp1)
    
Let's see how the loss function was changing during the training. We expect it to get smaller and smaller with every next epoch.

In [ ]:
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.plot(training_history.history['loss'], label='training set')
plt.plot(training_history.history['val_loss'], label='validation set')
plt.legend()

### 6.2 | Accuracy plot curve for training and validation<a class="anchor" id ="accuracyPlot"><a/>
[Back to Table of Contents](#bcImp1)

In [ ]:
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.plot(training_history.history['accuracy'], label='training set')
plt.plot(training_history.history['val_accuracy'], label='validation set')
plt.legend()

### 6.3 | Evaluation of the model accuracy<a class ="anchor" id ="accuracyEvaluation"></a>
[Back to Table of Contents](#modelAnchor)

We need to compare the accuracy of our model on training set and on valiation set. We expect our model to perform similarly on both sets. If the performance on a validation set will be poor comparing to a training set it would be an indicator for us that the model is overfitted and we have a "high variance" issue.

### 6.3.1 | Performance of training dataset <a class ="anchor" id ="perfTrain"></a>
[Back to Table of Contents](#modelAnchor)

In [ ]:
%%capture
train_loss, train_accuracy = model.evaluate(x_train_normalized, y_train_re)

In [ ]:
print('Train loss: ', train_loss)
print('Train accuracy: ', train_accuracy)

### 6.3.2 | Performance of validation dataset <a class ="anchor" id ="perfValidation"></a>
[Back to Table of Contents](#modelAnchor)

In [ ]:
%%capture
validation_loss, validation_accuracy = model.evaluate(x_validation_normalized, y_validation_re)

In [ ]:
print('Validation loss: ', validation_loss)
print('Validation accuracy: ', validation_accuracy)

With a validation score of close to 99%, we proceed to use this model to predict for the test set.

### 6.3.3 | Save and load the model<a class ="anchor" id ="saveModel"></a>
[Back to Table of Contents](#modelAnchor)

We will save the entire model to a HDF5 file. The .h5 extension of the file indicates that the model shuold be saved in Keras format as HDF5 file.

In [ ]:
model_name = 'digits_recognition_cnn.h5'
model.save(model_name, save_format='h5')

In [ ]:
loaded_model = tf.keras.models.load_model(model_name)

### 6.3.4 | Visualise validation predicted data on how the digits were written<a class ="anchor" id ="visualizevalidation"></a>
[Back to Table of Contents](#modelAnchor)

To use the model that we've just trained for digits recognition we need to call predict() method.

In [ ]:
predictions_one_hot = loaded_model.predict([x_validation_normalized])

In [ ]:
print('predictions_one_hot:', predictions_one_hot.shape)

Each prediction consists of 10 probabilities (one for each number from 0 to 9). We need to pick the digit with the highest probability since this would be a digit that our model most confident with.

In [ ]:
# Predictions in form of one-hot vectors (arrays of probabilities).
pd.DataFrame(predictions_one_hot)

In [ ]:
# Let's extract predictions with highest probabilites and detect what digits have been actually recognized.
predictions = np.argmax(predictions_one_hot, axis=1)
pd.DataFrame(predictions)

So our model is predicting that the first example from the validation set is 1.

Let's print the first image from a validation set to see if model's prediction is correct.

In [ ]:
plt.imshow(x_validation_normalized[0].reshape((IMAGE_WIDTH, IMAGE_HEIGHT)), cmap=plt.cm.binary)
plt.show()

We see that our model made a correct prediction and it successfully recognized digit 1. Let's print some more test examples and correspondent predictions to see how model performs and where it does mistakes.

In [ ]:
numbers_to_display = 196
num_cells = math.ceil(math.sqrt(numbers_to_display))
plt.figure(figsize=(15, 15))

for plot_index in range(numbers_to_display):
    predicted_label = predictions[plot_index]
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    color_map = 'Greens' if predicted_label == y_validation_re[plot_index] else 'Reds'
    plt.subplot(num_cells, num_cells, plot_index + 1)
    plt.imshow(x_validation_normalized[plot_index].reshape((IMAGE_WIDTH, IMAGE_HEIGHT)), cmap=color_map)
    plt.xlabel(predicted_label)

plt.subplots_adjust(hspace=1, wspace=0.5)
plt.show()

### 6.3.5 | Confusion matrix of validation dataset <a class = "anchor" id = "cmValidation"></a>
[Back to Table of Contents](#modelAnchor)

Confusion matrix shows what numbers are recognized well by the model and what numbers the model usually confuses to recognize correctly.

In [ ]:
confusion_matrix = tf.math.confusion_matrix(y_validation_re, predictions)
f, ax = plt.subplots(figsize=(9, 7))
sn.heatmap(
    confusion_matrix,
    annot=True,
    linewidths=.5,
    fmt="d",
    square=True,
    ax=ax
)
plt.show()

### In summary<a class="anchor" id ="modelEvaluationSummary"></a>
[Back to Table of Contents](#modelAnchor)

## <span style='color:rgb(205, 0, 153)'> 7 | Model prediction on unseen dataset (test data) </span><a class="anchor" id ="modelprediction"></a>
<hr>

[Back to Table of Contents](#modelAnchor)

In [ ]:
predictions_one_hot = loaded_model.predict([x_test_normalized])
print('predictions_one_hot:', predictions_one_hot.shape)

Each prediction has ten probabilities (one for each number from 0 to 9). We need to choose the digit with the highest probability.

#### Predicted probabilities of all digits.

In [ ]:
pd.DataFrame(predictions_one_hot)

### 7.1 | Visualise test predicted data how the digits were written<a class="anchor" id="visualizePredict"></a>
[Back to Table of Contents](#modelAnchor)

Predicted digits with highest probabilites

#### Actual first digit from the test data

In [ ]:
plt.imshow(x_test_normalized[0].reshape((IMAGE_WIDTH, IMAGE_HEIGHT)), cmap=plt.cm.binary)
plt.show()

So our model is predicting that the first example from the test data is 2.

### <span style='color:rgb(205, 0, 153)'>8 | Submission </span> <a class ="anchor" id ="submission"></a>
<hr>

[Back to Table of Contents](#modelAnchor)

In [ ]:
test_pred = pd.DataFrame( loaded_model.predict([x_test_normalized]))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

In [ ]:
test_pred.to_csv('submission.csv', index = False)

<span style='color:rgb(205, 0, 153)'>__Many thanks for reading my kernal!__🙏

<span style='color:rgb(205, 0, 153)'>__Please leave a comment if you have any suggestions for improving the analysis!__🏋🥇

<span style='color:rgb(0,204,0)'>__If you liked 😊 my kernel, give 👍 UPVOTE!__

<span style='color:rgb(205, 0, 153)'>__If you have a moment, I encourage you to see my other [kernels](https://www.kaggle.com/itsmohammadshahid/code?scroll=true).__